In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import f_oneway
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import f1_score, make_scorer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from operator import itemgetter
from joblib import dump, load

In [2]:
df = pd.read_csv("datasets/cleaned/datacleaned.csv")
df.head()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_6,gene_7,gene_8,gene_9,gene_10,...,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530,sample_id,Class
0,0.0,2.017209,3.265527,5.478487,10.431999,7.175175,0.591871,0.0,0.0,0.591871,...,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0,sample_0,3
1,0.0,0.592732,1.588421,7.586157,9.623011,6.816049,0.000000,0.0,0.0,0.000000,...,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0,sample_1,2
2,0.0,3.511759,4.327199,6.881787,9.870730,6.972130,0.452595,0.0,0.0,0.000000,...,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0,sample_2,3
3,0.0,3.663618,4.507649,6.659068,10.196184,7.843375,0.434882,0.0,0.0,0.000000,...,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0,sample_3,3
4,0.0,2.655741,2.821547,6.539454,9.738265,6.566967,0.360982,0.0,0.0,0.000000,...,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0,sample_4,0


In [3]:
df.set_index("sample_id", inplace = True)

In [4]:
X = df.drop(columns = ['Class'])
y = df['Class']

## Train - Test Split

To build a classification model, we must set aside certain proportion of our dataset for testing the performance of the model. However, there is a tricky issue with the current dataset. This gene expression dataset is very noisy. It has been found that there are genes which are only expressed in 20 percent or less samples. This means when we split the data n number of times, there might be certain splits where all the 0s fall in the training set, and all expression in the test set respectively. So, when we standardize the data using parameters of the training set, we might get a division by 0 on both the training and testing sets since the standard deviation of the genes in training set would be 0. 

Now since the sample set is relatively small, we must only choose the most important of all the informative features, and those features must be found only using the training set. For genes whose expression is mostly 0, there are 2 possibilities. Either the gene is expressed equally in all classes, in which case it is noise, or it is differentially expressed, in which case its relative importance wrt to other genes must be considered. But given the training size, we might end up completely ignoring these genes. 

In order to keep them, we must find out the maximum proportion of 0s in significant genes and split the data so that it doesn't exclude these genes automatically, except if this proportion is larger than 0.85. In order to give them a fair chance, we can do the following: First run ANOVA on the entire dataset to find out all differentially expressed genes, then find out the proportion of their zeros and split at the maximum proportion of zeros. That is, the splitting size 

s = min(0.85, max(proportion of zeros of all informative genes))

This will ensure that training set contains at least one sample of such genes and so can be standardized and be directly fed to week 2 analysis of dimensionality reduction as well as wrapper based feature selection algorithms. If there are genes which are zero on more than 85% of samples and still significant, then those genes can be studied separately and will not be considered for classification model as they would only create the dataset noisy in different iterations. 

In [5]:
def F_test(gene, labels):
    
    """
    *************************************************
    
    Method to check if a given gene varies significantly across samples of different cancer types by using ANOVA test
    
    params:
    gene: numpy array of expression of a given gene across all samples
    labels: numpy array of corresponding labels of cancer type
    
    returns:
    f: F-statistic
    p: p-value
    
    """
    #convert to numpy arrays
    genes = gene.values
    labels = labels.values
    #obtain expression of genes at different conditions
    genes0 = genes[np.where(labels==0)]
    genes1 = genes[np.where(labels==1)]
    genes2 = genes[np.where(labels==2)]
    genes3 = genes[np.where(labels==3)]
    genes4 = genes[np.where(labels==4)]
    #apply one way anova test
    f,p = f_oneway(genes0,genes1,genes2,genes3,genes4)
    
    return f,p

In [6]:
def countzeros(gene):
    """
    counts zeros in a gene expression
    """
    m = gene[gene==0].shape[0]
    return m

In [7]:
cols = X.columns.to_list()
significant_genes = [] #significant genes on the entire set
n = X.shape[0]
max_zero_prop = 0 # maximum proportion of zeros
minimally_expressed_genes = []
genes_to_remove = []

for i in cols:
    f,p = F_test(X[i],y)
    m = countzeros(X[i])
    zero_prop = m/n
    if zero_prop > 0.85:
        genes_to_remove.append(i)
    if(p<0.05):
        significant_genes.append((f,i, zero_prop))
        
        if zero_prop > 0.85:
            minimally_expressed_genes.append(i) 
        
        if zero_prop > max_zero_prop:
            max_zero_prop = zero_prop
    
print(f"maximum zero proportion in differentially expressed genes = {max_zero_prop}")

maximum zero proportion in differentially expressed genes = 0.9975031210986267


In [8]:
test_size = 1 - min(0.85,max(0.80,round(max_zero_prop*100 + 1)/100)) #test size shouldn't be greater than 0.2
print(f"Test size = {test_size}")

Test size = 0.15000000000000002


In [9]:
X1 = X.copy()
X1 = X1.drop(columns = genes_to_remove)
print(f"Total No. of genes dropped : {len(genes_to_remove)}")
print(f"No. of differentially expressed genes dropped : {len(minimally_expressed_genes)}")

Total No. of genes dropped : 1252
No. of differentially expressed genes dropped : 701


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X1,y,test_size=test_size, random_state=10, stratify=y)

In [11]:
train = X_train.copy()
train['Class'] = y_train
train.to_csv("datasets/cleaned/traindata.csv")

In [12]:
test = X_test.copy()
test['Class'] = y_test
test.to_csv("datasets/cleaned/testdata.csv")

# Feature Selection  

## ANOVA (Filter based)

In [13]:
#perform ANOVA on training set
traincols = X_train.columns.tolist()
important_genes = []

for i in traincols:
    f,p = F_test(X_train[i],y_train)
    if(p<0.05):
        important_genes.append((f,i))
        
len(important_genes)

18813

In [14]:
#sort important genes by F statistic: Higher F-statistic => lower p-value => more significant
important_genes = sorted(important_genes, key=itemgetter(0), reverse=True)
# select 500 most significant genes
informative_genes = [gene for f,gene in important_genes[:500]] 

## Stepwise Selection (Wrapper based)

As p is very large (~20,264), stepwise selection for the entire feature space would be computationally very expensive (training 1 + p(p+1)/2 = 205,324,981 models for each method!!), therefore, we would use only the 500 informative features selected from ANOVA to perform each of Forward Selection and Backward Elimination. 

For both of the methods, we'd be using sklearn's SequentialFeatureSelector, select top 350 features so that both methods return atleast 200 common features. 

We'd also need a scoring method. F1 score is preferred because of class imbalance, however we'd need to define it custom as follows. 

In [21]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFSM

In [15]:
f1_scorer = make_scorer(f1_score, average='weighted')

### Support Vector Classifier

#### Forward Selection

In [43]:
try:
    forwardm = load("week1/models/forwardsvcm.joblib")
except:
    #prepare input data
    X_fsm = X_train[informative_genes]
    y_fsm = y_train.copy()
    #define SFS model with Support Vector Classifier
    pipefm = Pipeline([('scaler', StandardScaler()), ('estimator', SVC(kernel='linear'))])
    forwardm = SFSM(pipefm, k_features=350, forward = True, scoring=f1_scorer, verbose=2, n_jobs=4)
    #fit the SFS method
    forwardm.fit(X_fsm,y_fsm)
    #save the model
    dump(forwardm,"week1/models/forwardsvcm.joblib")
    

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    3.9s
[Parallel(n_jobs=4)]: Done 272 tasks      | elapsed:    8.7s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:   13.8s finished

[2022-03-12 16:56:01] Features: 1/350 -- score: 0.7377746998090775[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  92 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 499 out of 499 | elapsed:    6.8s finished

[2022-03-12 16:56:08] Features: 2/350 -- score: 0.9292709767207553[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  92 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done 491 out of 498 | elapsed:    5.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done 498 out of 498 | elapsed:    5.0s finished

[2022-03-12 16:56:13] Features: 3/350 -- score: 0.9648366097888633[Parallel(n_jobs=4)]: Using backend Loky

**It can be seen that after 8 features validation score stopped increasing, indicating that features are redundant. Since mlxtend or sklearn don't provide earlystopping, the searching was manually stopped.** 

#### Backward Elimination

In [45]:
try:
    backwardm = load("week1/models/backwardsvcm.joblib")
except:
    #prepare the data
    X_bem = X_train[informative_genes]
    y_bem = y_train.copy()
    #define SFS model with Support Vector Classifier
    pipebm = Pipeline([('scaler', StandardScaler()), ('estimator', SVC(kernel='linear'))])
    backwardm = SFSM(pipebm, k_features=350, forward = False, scoring = f1_scorer, verbose=2, n_jobs=4)
    #fit the model
    backwardm.fit(X_bem,y_bem)
    #save the model
    dump(backwardm, "week1/models/backwardsvcm.joblib")

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    4.7s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    9.9s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:   18.6s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:   24.8s finished

[2022-03-12 17:19:24] Features: 499/350 -- score: 0.9985209059868525[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  58 tasks      | elapsed:    2.5s
[Parallel(n_jobs=4)]: Done 300 tasks      | elapsed:   12.9s
[Parallel(n_jobs=4)]: Done 492 out of 499 | elapsed:   20.7s remaining:    0.2s
[Parallel(n_jobs=4)]: Done 499 out of 499 | elapsed:   21.0s finished

[2022-03-12 17:19:45] Features: 498/350 -- score: 0.9985209059868525[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  58 tasks      | elapsed:    2.5s
[Parallel(n_jobs=4)]: Done 300 tasks      | elapsed

### RandomForest Classifier

#### Forward Selection

In [33]:
try:
    forward_rf = load("week1/models/forwardrf.joblib")
except:
    #prepare input data
    X_fs_rf = X_train[informative_genes]
    y_fs_rf = y_train.copy()
    #define SFS model with RandomForest Classifier
    pipef_rf = Pipeline([('scaler', StandardScaler()), ('estimator', RandomForestClassifier(n_estimators=200,random_state=1))])
    forward_rf = SFSM(pipef_rf, k_features=350, forward = True, scoring=f1_scorer, verbose=2, n_jobs=4)
    #fit the SFS method
    forward_rf.fit(X_fs_rf,y_fs_rf)
    #save the model
    dump(forward_rf,"week1/models/forwardrf.joblib")

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   19.9s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:  3.3min
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:  4.6min finished

[2022-03-12 15:18:35] Features: 1/350 -- score: 0.6989024297847701[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   18.8s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:  3.2min
[Parallel(n_jobs=4)]: Done 499 out of 499 | elapsed:  4.5min finished

[2022-03-12 15:23:06] Features: 2/350 -- score: 0.9184096652246623[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   18.4s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  1.4min
[Parallel(n_j

### Stepwise Selection : Analysis

Again, it can be seen that removing features doesn't change validation scores, strongly indicating that useful feature set is much smaller and the total feature set contains lot of redundancy, i.e. features which do not contribute in any way. If we pre-specify the number of features to select, the algorithm will start listing all the redundant features in order to select the given number of features. **Therefore, search was manually stopped, and this method is discarded.** 

However, this application gives us new information about the data: If we look at f1 scores of the above 2 method, both the times it reaches 0.9985 and stops changing, indicating that features selected from ANOVA are very informative, and before feeding our data to these algorithms, we have succesfully removed most of the noise. It further suggests that we need a mechanism which looks at the bulk of the features and removes all the redundant features. 

In their landmark paper Guyon, I., Weston, J., Barnhill, S., & Vapnik, V., *“Gene selection for cancer classification using support vector machines”* propose a new method of gene selection utilizing Support Vector Machine methods based on **Recursive Feature Elimination (RFE)**. It's implementation is provided by scikit-learn's RFECV class. RFECV from scikit-learn recursively drops the least important features based on a model's `coef_` or `feature_importance_` attribute, after cross validating. The fraction of features to drop at each iteration is decided by the `step` parameter. Each of the resulting feature set is considered as a subset and is assigned a cross-validation score which is available in the attribute `grid_scores_`.

## Recursive Feature Elimination 

### Using Support Vector Classifier

In [60]:
try:
    recursive_svcm = load("week1/models/recursive_svcm.joblib")
except:
    #prepare data
    X_rfe_svcm = X_train[informative_genes].copy()
    y_rfe_svcm = y_train.copy()
    #define RFECV with SVC model
    pipersvcm = Pipeline([('scaler', StandardScaler()), ('clf', SVC(kernel='linear'))])
    recursive_svcm = RFECV(pipersvcm,step = 0.02, scoring = f1_scorer, importance_getter=lambda x:x[-1].coef_)
    #fit the model
    recursive_svcm.fit(X_rfe_svcm,y_rfe_svcm)
    #save the model
    #dump(recursive_svcm,"week1/models/recursive_svcm.joblib")

In [61]:
#get selected features
selected_indices_recursive_svcm = recursive_svcm.get_support(indices=True)
selected_feat_recursive_svcm = []
for i,col in enumerate(X_rfe_svcm.columns.to_list()):
    if i in selected_indices_recursive_svcm:
        selected_feat_recursive_svcm.append(col)

In [83]:
recursive_svcm.score(X_rfe_svcm,y_rfe_svcm)

1.0

### Using Random Forest Classifier

In [73]:
try:
    recursive_rfm = load("week1/models/recursive_rfm.joblib")
except:
    #prepare data
    X_rfe_rfm = X_train[informative_genes].copy()
    y_rfe_rfm = y_train.copy()
    #define RFECV with SVC model
    piperrfm = Pipeline([('scaler', StandardScaler()), ('estimator', RandomForestClassifier(n_estimators=100))])
    recursive_rfm = RFECV(piperrfm,step = 0.1, scoring = f1_scorer, importance_getter=lambda x: x[1].feature_importances_)
    #fit the model
    recursive_rfm.fit(X_rfe_rfm,y_rfe_rfm)
    #save the model
    #dump(recursive_rfm,"week1/models/recursive_rfm.joblib")

In [74]:
#get selected features
selected_indices_recursive_rfm = recursive_rfm.get_support(indices=True)
selected_feat_recursive_rfm = []
for i,col in enumerate(X_rfe_rfm.columns.to_list()):
    if i in selected_indices_recursive_rfm:
        selected_feat_recursive_rfm.append(col)

In [86]:
recursive_rfm.score(X_rfe_rfm,y_rfe_rfm)

1.0

## Selecting common features

In [87]:
len(selected_feat_recursive_svcm)

90

In [89]:
len(selected_feat_recursive_rfm)

100

In [91]:
common_feat = list(set(selected_feat_recursive_svcm).intersection(set(selected_feat_recursive_rfm)))

In [92]:
len(common_feat)

47

In [94]:
trainingdata = X_train[common_feat].copy()
trainingdata['Class'] = y_train
testingdata = X_test[common_feat].copy()
testingdata['Class'] = y_test

In [95]:
trainingdata.to_csv("datasets/featureselection/train.csv")
testingdata.to_csv("datasets/featureselection/test.csv")